This is a notebook to pull down all the information for figure 3 (ncbi hits). The design is to compair between each sample (barcode) instead of each flowcell as they sequenced the same thing. Therefore I just pulled all the ncbi hits for each flowcell for each barcode and merged them together according to the barcode.

In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import argparse
from ete3 import NCBITaxa 

In [2]:
ncbi = NCBITaxa()

In [3]:
# use argparse to do this
#BASEDIR = args.BASEDIR
BASEDIR = '/home/yiheng/data/20170617_FAH05432'

# here we define the folder name of the dataframe it created by capturing the folder from the BASDIR
folder_name = os.path.basename(BASEDIR)
column_name = folder_name.split('_')[-1]

In [4]:
# first check if the analysis folder is there
folder_list = 'analysis  basecalled_data  scripts  tracking  workspace'.split(' ')
for x in range(0,folder_list.count('')):
    folder_list.remove('')
#fix this test
if not set(os.listdir(os.path.abspath(BASEDIR))) == set (folder_list):
    print("Something wrong with basefolder. check it please.")

In [5]:
# get the dataframe there
dataframe = os.path.join(BASEDIR, 'analysis', 'summary_df_%s.tab' % folder_name)
sum_df = pd.read_csv(dataframe, sep='\t')

/home/yiheng/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,2,3,5,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
sum_df.columns

Index(['read_id', 'passes_filtering', 'sequence_length_template',
       'mean_qscore_template', 'barcode_arrangement', 'barcode_score', 'kit',
       'variant', 'pc_survived', 'nl_survived', 'qseqid_rg', 'sseqid_rg',
       'evalue_rg', 'length_rg', 'pident_rg', 'nident_rg', 'sacc_rg',
       'staxids_rg', 'scomnames_rg', 'read_length_pc_x', 'qseqid_nt',
       'sseqid_nt', 'evalue_nt', 'length_nt', 'pident_nt', 'nident_nt',
       'sacc_nt', 'staxids_nt', 'scomnames_nt', 'read_length_pc_y'],
      dtype='object')

In [7]:
# fills the nan with T/F for easier handling
sum_df.sseqid_rg.fillna(False, inplace=True)
sum_df.sseqid_nt.fillna(False, inplace=True)
# filter out the rg blast hit
ntblasthit_reads = sum_df[(sum_df.sseqid_rg == False) & (sum_df.sseqid_nt != False) & (sum_df.passes_filtering == True) & (sum_df.pc_survived == True) & (sum_df.nl_survived == True)]
pd.set_option('display.max_columns', None)

#Removing unncessary columns from joint_NCBI_df_taxonomy
#I hard coded to select as it depends on what information you need
for columns in ['passes_filtering', 'sequence_length_template',
       'mean_qscore_template', 'barcode_score', 'kit',
       'variant', 'pc_survived', 'nl_survived', 'qseqid_rg', 'sseqid_rg',
       'evalue_rg', 'length_rg', 'pident_rg', 'nident_rg', 'sacc_rg',
       'staxids_rg', 'scomnames_rg', 'read_length_pc_x', 'qseqid_nt',
       'sseqid_nt', 'evalue_nt', 'length_nt', 'pident_nt', 'nident_nt',
       'sacc_nt','scomnames_nt', 'read_length_pc_y']:
    del ntblasthit_reads[columns] 

In [8]:
###########This block for barcodes order
# now everything left is what we want to plot out
# arrange different barcodes
# REMEMBER: This need to be manual check:
# barcode01 is Pst infected sample, barcode02 is Zymo sample, 03 is nill sample, 04 is co-infection sample and 05 is Pyre sample
new_barcodes = ['barcode01', 'barcode02', 'barcode03', 'barcode04', 'barcode05']
if column_name == 'run1':
    ntblasthit_barcode01 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode02']
    ntblasthit_barcode02 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode06']
    ntblasthit_barcode03 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode04']
    ntblasthit_barcode04 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode05']
    ntblasthit_barcode05 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode03']
    ntblasthit_barcode01.barcode_arrangement = ntblasthit_barcode01.barcode_arrangement.replace(to_replace='barcode02', value='barcode01')
    ntblasthit_reads.drop(ntblasthit_reads.index[ntblasthit_reads.barcode_arrangement.str.contains('barcode07')], inplace = True)
    ntblasthit_barcode01.barcode_arrangement = ntblasthit_barcode01.barcode_arrangement.replace(to_replace='barcode03', value='barcode07')
    ntblasthit_barcode01.barcode_arrangement = ntblasthit_barcode01.barcode_arrangement.replace(to_replace='barcode06', value='barcode02')
    ntblasthit_barcode01.barcode_arrangement = ntblasthit_barcode01.barcode_arrangement.replace(to_replace='barcode04', value='barcode03')
    ntblasthit_barcode01.barcode_arrangement = ntblasthit_barcode01.barcode_arrangement.replace(to_replace='barcode05', value='barcode04')
    ntblasthit_barcode01.barcode_arrangement = ntblasthit_barcode01.barcode_arrangement.replace(to_replace='barcode07', value='barcode05')

elif column_name == 'run2':
    # first delete the rows that contains the barcode we are going to replace
    # I think maybe better to first adjust the df to get rid of all the mis/unclassified reads so the df get smaller
    # and we do not need those reads for plotting
    
    ntblasthit_reads.drop(ntblasthit_reads.index[ntblasthit_reads.barcode_arrangement.str.contains('barcode01')], inplace = True)
    ntblasthit_reads.drop(ntblasthit_reads.index[ntblasthit_reads.barcode_arrangement.str.contains('barcode02')], inplace = True)
    ntblasthit_reads.drop(ntblasthit_reads.index[ntblasthit_reads.barcode_arrangement.str.contains('barcode03')], inplace = True)
    ntblasthit_reads.drop(ntblasthit_reads.index[ntblasthit_reads.barcode_arrangement.str.contains('barcode04')], inplace = True)
    ntblasthit_reads.drop(ntblasthit_reads.index[ntblasthit_reads.barcode_arrangement.str.contains('barcode05')], inplace = True)
    ntblasthit_reads.drop(ntblasthit_reads.index[ntblasthit_reads.barcode_arrangement.str.contains('unclassified')], inplace = True)
    old_barcodes = ['barcode07', 'barcode08', 'barcode09', 'barcode10', 'barcode11']
    for old_barcode, new_barcode in zip(old_barcodes, new_barcodes):
        ntblasthit_reads.loc\
        [ntblasthit_reads[ntblasthit_reads['barcode_arrangement'] == old_barcode].index,'barcode_arrangement']\
        = [new_barcode]
    #ntblasthit_reads['barcode_arrangement'] = ntblasthit_reads.barcode_arrangement.replace(to_replace='barcode07', value='barcode01')

    
elif column_name == 'FAH05432':
    ntblasthit_barcode01 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode01']
    ntblasthit_barcode02 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode02']
    ntblasthit_barcode03 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode03']
    ntblasthit_barcode04 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode04']
    ntblasthit_barcode05 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode05']
elif column_name == 'FAH05731':
    ntblasthit_barcode01 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode01']
    ntblasthit_barcode02 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode02']
    ntblasthit_barcode03 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode03']
    ntblasthit_barcode04 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode04']
    ntblasthit_barcode05 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode05']
else:
    pass



In [9]:
# now just filter out all the reads that are not going to used for ploting
ntblasthit_reads_barcode01 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement.str.contains('barcode01')]
ntblasthit_reads_barcode02 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement.str.contains('barcode02')]
ntblasthit_reads_barcode03 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement.str.contains('barcode03')]
ntblasthit_reads_barcode04 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement.str.contains('barcode04')]
ntblasthit_reads_barcode05 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement.str.contains('barcode05')]

# now concat them together. but remember that the index has to change to add the column for ncbi taxa
ntblasthit_reads_filtered_barcodes = pd.concat([ntblasthit_reads_barcode01, 
                                                ntblasthit_reads_barcode02, 
                                                ntblasthit_reads_barcode03, 
                                                ntblasthit_reads_barcode04, 
                                                ntblasthit_reads_barcode05], ignore_index=True)

In [10]:
ntblasthit_reads_filtered_barcodes

,read_id,barcode_arrangement,staxids_nt
0,807a8997-b021-4820-b33a-1b9095249dc7,barcode01,200451
1,c53fa9c8-f4d5-4876-8526-e04fcea0ac5d,barcode01,200451
2,19a62259-0925-41ef-94e4-6fe9ef776c81,barcode01,199201
3,7e4e83ba-26fa-4d0b-b714-724976a9c7a7,barcode01,200451
4,c92e3d57-c554-4d73-9e17-a51bc14aa4d4,barcode01,1.35728e+06
5,072bbc16-20a5-45b8-bcbf-e7239f48ed12,barcode01,1.32493e+06
6,7eb68975-bd44-40da-b0e8-2635fcedfbb0,barcode01,716540
7,f876c268-fe76-4fb7-97e7-e3c6fa2d8c8b,barcode01,1.28236e+06
8,00f1b4d2-cc23-4231-9b51-543873ddddd2,barcode01,634500
9,74885d13-7285-4de6-ae85-89a8927c36ab,barcode01,1.50567e+06


In [11]:
# as the concate before will result the staxid into a string
# so now change the string back to float so it can be recognized by the NCBITaxa
for taxid in ntblasthit_reads_filtered_barcodes['staxids_nt']:
    if ';' in str(taxid):
        taxid = taxid.split(';')[0]
    else:
        pass
    float(taxid)

In [12]:
#Function to generate taxonomy columns based on NCBITaxa results for NCBI hit dataframe, 
def search_rank_output_name_append_column(df, staxid_column, rank_search):
    """Input df, staxid_column from same df and rank_search (a desired taxonomic rank 
    from each staxid's lineage), outputs taxonomic name corresponding to rank_search or 'Unclassified' if
    unavailable and appends to df row by row"""
    rank_list = []
    for read_index in range(0, len(staxid_column)):
        taxid = ''
        if ';' in str(staxid_column[read_index]):
            taxid = staxid_column[read_index].split(';')[0]
        else:
            taxid = staxid_column[read_index]
        
        taxid_lineage = ''
        taxid_lineage = ncbi.get_lineage(taxid)
        
        names = ''
        names = ncbi.get_taxid_translator(taxid_lineage)
        
        ranks = ''
        ranks = ncbi.get_rank(taxid_lineage) #Dict
        
        ranks2names = ''
        ranks2names = {ranks[k]:names[k] for k in names.keys() & ranks}
        
        if rank_search in ranks2names.keys():
            rank_list.append(ranks2names[rank_search])#if rank in dict, print name
        else:
            rank_list.append('Unclassified')
    df[rank_search] = rank_list
# NOTE: Appending is always slow, try and find a better way e.g df.apply to a column based on staxids column


In [13]:
#ntblasthit_reads_filtered_barcodes_added_TaxaRank = ntblasthit_reads_filtered_barcodes.copy()
rank_list = ['superkingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']
for rank in rank_list:
    search_rank_output_name_append_column(ntblasthit_reads_filtered_barcodes, 
                                          ntblasthit_reads_filtered_barcodes.staxids_nt, 
                                          rank)

In [14]:
ntblasthit_reads_filtered_barcodes

,read_id,barcode_arrangement,staxids_nt,superkingdom,phylum,class,order,family,genus,species
0,807a8997-b021-4820-b33a-1b9095249dc7,barcode01,200451,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Pseudomonas,Pseudomonas poae
1,c53fa9c8-f4d5-4876-8526-e04fcea0ac5d,barcode01,200451,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Pseudomonas,Pseudomonas poae
2,19a62259-0925-41ef-94e4-6fe9ef776c81,barcode01,199201,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Pseudomonas,Pseudomonas syringae
3,7e4e83ba-26fa-4d0b-b714-724976a9c7a7,barcode01,200451,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Pseudomonas,Pseudomonas poae
4,c92e3d57-c554-4d73-9e17-a51bc14aa4d4,barcode01,1.35728e+06,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Pseudomonas,Pseudomonas syringae
5,072bbc16-20a5-45b8-bcbf-e7239f48ed12,barcode01,1.32493e+06,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Pseudomonas,Pseudomonas syringae
6,7eb68975-bd44-40da-b0e8-2635fcedfbb0,barcode01,716540,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Erwiniaceae,Erwinia,Erwinia amylovora
7,f876c268-fe76-4fb7-97e7-e3c6fa2d8c8b,barcode01,1.28236e+06,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Pseudomonas,Pseudomonas poae
8,00f1b4d2-cc23-4231-9b51-543873ddddd2,barcode01,634500,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Erwiniaceae,Erwinia,Erwinia billingiae
9,74885d13-7285-4de6-ae85-89a8927c36ab,barcode01,1.50567e+06,Eukaryota,Basidiomycota,Pucciniomycetes,Pucciniales,Pucciniaceae,Puccinia,Puccinia cf. psidii AE-2014


In [15]:
# add the flowcell column just in case
ntblasthit_reads_filtered_barcodes = ntblasthit_reads_filtered_barcodes.assign(Flowcell = column_name)

In [16]:
ntblasthit_reads_filtered_barcodes.to_csv(r'/home/yiheng/analysis/WGS/%s_taxa.tab' % column_name, header=column_name, index=None, sep='\t')

In [ ]:
df1=pd.DataFrame([[1, 2],[10, 1], [3, 4], [7, 8]], columns=['A', 'B'])
df2=pd.DataFrame([[1, 4],[3, 5]], columns=['A', 'B'])
# now replace the location use the index of each df selected from value by .loc function
# first select the value from df2 column A
for values in df2['A'].unique():
    # then locate the position in df1 and replace it by the VALUE of the position in df2
    df1.loc[df1[df1['A'] == (values)].index, 'B'] = df2.loc[df2[df2['A'] == (values)].index, 'B'].values

In [ ]:
df1